In [2]:
from astropy.timeseries import LombScargleMultiband
import numpy as np
import matplotlib.pyplot as pyplot

%matplotlib inline

In [5]:
def GeneratePeriodicData(n=10, period=10):
    """Generate periodic data for testing Lomb Scargle Periodogram.

    Parameters
    ----------
    n : int
        Number of data points.
    period : float
        Period of the periodic signal.

    Returns
    -------
    t : np.ndarray
        Time values.
    y_obs : np.ndarray
        Observed flux values.
    """
    np.random.seed(42)

    t = np.random.randint(100, size=n) + 0.3 + 0.4 * np.random.random(n)
    y = 10 + np.sin(2 * np.pi * t / period)
    dy = 0.001 + 0.001 * np.random.random(n)
    y_obs = np.random.normal(y, dy)

    return t, y_obs

In [11]:
n_sources = 10 
time, flux = GeneratePeriodicData()
bands = n_sources * ["u"]
yerr = 1e-3+np.zeros(n_sources)

In [12]:
def compute_optimized_periodogram_grid(x0, oversampling_factor=5, nyquist_factor=100):
    """
    Computes an optimized periodogram frequency grid for a given time series.

    Parameters
    ----------
    x0 : `array`
        The input time axis.
    oversampling_factor : `int`, optional
        The oversampling factor for frequency grid.
    nyquist_factor : `int`, optional
        The Nyquist factor for frequency grid.

    Returns
    -------
    frequencies : `array`
        The computed optimized periodogram frequency grid.
    """

    num_points = len(x0)
    baseline = np.max(x0) - np.min(x0)

    # Calculate the frequency resolution based on oversampling factor and baseline
    frequency_resolution = 1. / baseline / oversampling_factor

    num_frequencies = int(
        0.5 * oversampling_factor * nyquist_factor * num_points)
    frequencies = frequency_resolution + \
        frequency_resolution * np.arange(num_frequencies)

    return frequencies

In [14]:
lsp = LombScargleMultiband(time, flux, bands, dy=yerr,
                            nterms_base=1, nterms_band=1)

In [16]:
f_grid = compute_optimized_periodogram_grid(
    time, oversampling_factor=5, nyquist_factor=100)
period = 1/f_grid
power = lsp.power(f_grid)

In [20]:
period[np.argmax(power)].round(6)

9.994515

In [21]:
power[np.argmax(power)].round(6)

0.99983